In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # to disable GPU
from ts_data.ts_data import ts_data as ts
import getStocks
import visualize
import ts_models.predicts
import pandas as pd
import ts_models.ts_lstm as ts_lstm

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [ ]:
# view tenslow enabled devices
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

# User parameters

In [ ]:
entityID = 'amd'.upper()
n_in = 360 # number of timesteps as input
n_out = 360 # number of timesteps as output
target = 'close_AMD'

# Load Data

In [ ]:
dataset = pd.read_csv('/mnt/ubudata/projects/data/AMD_plus_100.csv')
# dataframe should be in INCREASING chronological order

In [ ]:
dataset.head()

In [ ]:
# ie. row 1 is Jan 1, row 2 in Jan 2.
dataset.sort_values('dtg', inplace=True)
# and do a forward fill
dataset.fillna(method='ffill', inplace=True)
# then drop the NA
dataset.dropna(inplace=True)

In [ ]:
# rename dtg to date for TS package
dataset.rename(columns={'dtg':'Date'}, inplace=True)

In [ ]:
dataset.head()

In [ ]:
dataset.shape

# Prepare Data for LSTM

In [ ]:
ts_data = ts(n_in=n_in, 
             n_out=n_out, 
             entityID=entityID,
             target=target,
             rawData=dataset)

In [ ]:
ts_data.eng_features(derivate=False, weekdays=False)

In [ ]:
ts_data.roll_data()

In [ ]:
# ts_data.data.to_csv('AMD_TS_DATA.csv',index=None)

In [ ]:
ts_data.tscv(train=0.98)

In [ ]:
ts_data.data.tail()

# Build LSTM Architecture

In [ ]:
ts_model = ts_lstm.lstm_model(ts_data, 
                               inlayer=int(ts_data.train_X.shape[-1])*2,
                               hiddenlayers=0,
                               loss_function='mae',
                               dropout=0.05,
                               activation='tanh',
                               gpus=1)

# Fit model

In [ ]:
import time
start = time.time()
history = ts_model.fit(ts_data.train_X, ts_data.train_y, 
                    epochs=50, 
                    batch_size=1024, 
                    validation_data=(ts_data.test_X, ts_data.test_y), 
                    verbose=2, 
                    shuffle=False)
fitTime = time.time()-start
print('Fit Time: {}'.format(round(fitTime,2)))

In [ ]:
visualize.plot_loss(history)

# Results

In [ ]:
# select the last sequence of data (it is length of n_in)
test_X = ts_data.test_X[-1]
test_Y = ts_data.test_y[-1]

In [ ]:
ts_data.data.shape

In [ ]:
test_X.reshape(1, test_X.shape[-2], test_X.shape[-1]).shape

In [ ]:
yhat = ts_model.predict(test_X.reshape(1, test_X.shape[-2], test_X.shape[-1]))

In [ ]:
import matplotlib.pyplot as plt
x = [x for x in range(len(yhat[0]))]
plt.plot(x, yhat[0], label='predicted')
plt.plot(x, test_Y, label='actual')
plt.title(entityID)
plt.xlabel('days')
plt.ylabel('amount change in price')
plt.legend()
plt.show()